In [6]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_wesad as dr_w
import tools.data_reader_ascertain as dr_asc
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)# IMPORTING MODULES

In [7]:
metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    # train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
]

model_phases_wesad = dr_w.Phases.PHASE_ORDER

wesad_label_type = "stai"
asc_label_type = dr_asc.SelfReports.AROUSAL

models = {
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "DT": DecisionTreeClassifier(criterion="gini", max_features="sqrt"),
    "LogReg": LogisticRegression(max_iter=1000),
    "RF": RandomForestClassifier(n_estimators=100, max_features="sqrt"),  #  min_samples_split=3
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="error")
}

threshold = "fixed"
asc_threshold = "fixed"
test_size = 1.0


In [8]:
# TRAIN ON ASCERTAIN AND TEST ON WESAD -- ALL
importlib.reload(train)
importlib.reload(dr_w)
importlib.reload(dt)


x_a, y_a = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold, normalize=True)
x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True)
# drop subjects with noisy data
inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# x = x[x['subject'] != 8.0]
# y = y[y['subject'] != 8.0]

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    "KNN": [],
    "DT": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
reports = {
    "KNN": [],
    "DT": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
num_iters = 1
get_importance = True
for _ in range(num_iters):
    out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=True, drop_subject=True)
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name][0])
        reports[model_name].append(out[model_name][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(e)
                # print(out[model_name][0][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print("")
print("\n")

y_train:
1    3001
0    1540
Name: label, dtype: int64
y_test:
0    779
1    180
Name: label, dtype: int64
Model KNN, Predictions: [0 1], [138 821]
Feature importance not available for KNN
Model DT, Predictions: [0 1], [497 462]
Model LogReg, Predictions: [0 1], [ 28 931]
Model RF, Predictions: [0 1], [273 686]
Model XGB, Predictions: [0 1], [473 486]

zip argument #2 must support iteration


[('rmssd', 0.16544746116276338), ('bpm', 0.15866712668565788), ('mean_SCL', 0.1516088190557134), ('sdnn', 0.14438110300769422), ('lf_hf_ratio', 0.12955667910526988), ('hf_rr', 0.12322377778815723), ('lf_rr', 0.09338436967577488), ('SCR_rate', 0.03373066351896915)]


[('SCR_rate', 0.32136064076511145), ('mean_SCL', 0.16795083666647767), ('hf_rr', -0.08106088974854966), ('lf_hf_ratio', -0.11289343501967576), ('bpm', -0.299761021414726), ('lf_rr', -0.33511538607025276), ('rmssd', -0.965536298357176), ('sdnn', -1.2139235835099742)]


[('mean_SCL', 0.1641458996702278), ('bpm', 0.1511182508282308), ('rm

In [9]:
# TRAIN ON ASCERTAIN AND TEST ON WESAD
importlib.reload(train)
importlib.reload(dr_w)
importlib.reload(dt)


# for j, phases_wesad in enumerate(model_phases_wesad):
for j, phases_wesad in enumerate([dr_w.Phases.TSST]):
    print(f"WESAD PHASES {phases_wesad} " + "-"*50)
    phases_wesad = [phases_wesad]
    x_a, y_a = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold, normalize=True)
    x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True)
    # drop subjects with noisy data
    inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
    x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
    y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)

    # x = x[x['subject'] != 8.0]
    # y = y[y['subject'] != 8.0]

    x_a = x_a.drop(["phaseId"], axis=1)
    x_b = x_b.drop(["phaseId"], axis=1)

    # make sure subjects from different datasets aren't labeled with the same index
    x_b["subject"] = x_b["subject"] + 500
    y_b["subject"] = y_b["subject"] + 500

    acc_results = {
        "KNN": [],
        "DT": [],
        "LogReg": [],
        "RF": [],
        "XGB": []
    }
    reports = {
        "KNN": [],
        "DT": [],
        "LogReg": [],
        "RF": [],
        "XGB": []
    }
    num_iters = 1
    get_importance = True
    for _ in range(num_iters):
        out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=True, drop_subject=True)
        for model_name in acc_results:
            acc_results[model_name].append(out[model_name][0])
            reports[model_name].append(out[model_name][1])
            if get_importance:
                try:
                    print("")
                    feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                    feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                    print(feature_imp)
                except Exception as e:
                    print(e)
                    # print(out[model_name][0][2])
                print("")

    for model_name in acc_results.keys():
        print(f"Model evaluation metrics for {model_name}:")
        for i in range(len(reports[model_name])):
            report = reports[model_name][i]
            acc = acc_results[model_name][i]
            p = report["precision"]
            r = report["recall"]
            f1 = report["f1"]
            auc = report["auc"]
            print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
        print("")
    print("\n")

WESAD PHASES TSST --------------------------------------------------
y_train:
1    3001
0    1540
Name: label, dtype: int64
y_test:
1    180
0     45
Name: label, dtype: int64
Model KNN, Predictions: [0 1], [ 98 127]
Feature importance not available for KNN
Model DT, Predictions: [0 1], [ 88 137]
Model LogReg, Predictions: [0 1], [ 52 173]
Model RF, Predictions: [0 1], [ 78 147]
Model XGB, Predictions: [0 1], [145  80]

zip argument #2 must support iteration


[('mean_SCL', 0.17648262883498966), ('sdnn', 0.1625647619268994), ('rmssd', 0.13933485431535958), ('bpm', 0.13904171834805956), ('hf_rr', 0.12966742002841392), ('lf_hf_ratio', 0.11998836353463783), ('lf_rr', 0.10344621087929798), ('SCR_rate', 0.02947404213234206)]


[('SCR_rate', 0.32136064076511145), ('mean_SCL', 0.16795083666647767), ('hf_rr', -0.08106088974854966), ('lf_hf_ratio', -0.11289343501967576), ('bpm', -0.299761021414726), ('lf_rr', -0.33511538607025276), ('rmssd', -0.965536298357176), ('sdnn', -1.2139235835099742)]



In [11]:
# TRAIN ON WESAD AND TEST ON ASCERTAIN
importlib.reload(train)
importlib.reload(dr_w)
importlib.reload(dt)


print(f"WESAD PHASES {model_phases_wesad} " + "-"*50)
x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True)
x_b, y_b = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold, normalize=True)
# drop subjects with noisy data
inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)

inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    "KNN": [],
    "DT": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
reports = {
    "KNN": [],
    "DT": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
num_iters = 1
get_importance = True
for _ in range(num_iters):
    out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=True, get_importance=get_importance, drop_subject=True)
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name][0])
        reports[model_name].append(out[model_name][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(e)
                # print(out[model_name][0][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print("")
print("\n")

WESAD PHASES ['Base', 'TSST', 'Medi_1', 'Fun', 'Medi_2'] --------------------------------------------------
Ratio of positive to negative labels (0.23106546854942234) is under 0.333, oversampling positive class.
y_train:
0    779
1    779
Name: label, dtype: int64
y_test:
1    3001
0    1540
Name: label, dtype: int64
Model KNN, Predictions: [0 1], [3419 1122]
Feature importance not available for KNN
Model DT, Predictions: [0 1], [3300 1241]
Model LogReg, Predictions: [0 1], [3348 1193]
Model RF, Predictions: [0 1], [3690  851]
Model XGB, Predictions: [0 1], [3435 1106]

zip argument #2 must support iteration


[('bpm', 0.3494885780502106), ('lf_rr', 0.20818061397966092), ('rmssd', 0.19608109798525689), ('SCR_rate', 0.09885350148091414), ('mean_SCL', 0.04880079486596081), ('hf_rr', 0.04031296318005555), ('lf_hf_ratio', 0.03693035347187229), ('sdnn', 0.021352096986068877)]


[('bpm', 11.169288732567502), ('rmssd', 3.6979516207007768), ('SCR_rate', 3.2559887979624613), ('mean_SCL', 1.7550